__Word Alignment Assignment__

Your task is to learn word alignments for the data provided with this Python Notebook. 

Start by running the 'train' function below and implementing the assertions which will fail. Then consider the following improvements to the baseline model:
* Is the TranslationModel parameterized efficiently?
* What form of PriorModel would help here? (Currently the PriorModel is uniform.)
* How could you use a Hidden Markov Model to model word alignment indices? (There's an implementation of simple HMM below to help you start.)
* How could you initialize more complex models from simpler ones?
* How could you model words that are not aligned to anything?

Grades will be assigned as follows*:

 AER below on blinds   |  Grade 
----------|-------------
 0.5 - 0.6 |   1 
 0.4 - 0.5 |   2 
 0.35 - 0.4 |  3    
 0.3 - 0.35 |  4    
 0.25 - 0.3 |  5   
 
You should save the notebook with the final scores for 'dev' and 'test' test sets.

*__Note__: Students who submitted a version of this assignment last year will have a 0.05 AER handicap, i.e to get a grade of 5, they will need to get an AER below 0.25.


In [1]:
# This cell contains the generative models that you may want to use for word alignment.
# Currently only the TranslationModel is at all functional.

import numpy as np
from copy import deepcopy
from collections import defaultdict

class TranslationModel:
    "Models conditional distribution over trg words given a src word."

    def __init__(self, src_corpus, trg_corpus):
        self._trg_given_src_probs = defaultdict(lambda : defaultdict(lambda : 1.0))
        self._src_trg_counts = defaultdict(lambda : defaultdict(lambda : 0.0))

    def get_params(self):
        return self._trg_given_src_probs

    def get_conditional_prob(self, src_token, trg_token):
        "Return the conditional probability of trg_token given src_token."
        return self._trg_given_src_probs[src_token][trg_token]

    def get_parameters_for_sentence_pair(self, src_tokens, trg_tokens):
        "Returns matrix with t[i][j] = p(f_j|e_i)."
        return np.array([[self._trg_given_src_probs[src_token][trg_token]
                          for trg_token in trg_tokens] for src_token in src_tokens])

    def collect_statistics(self, src_tokens, trg_tokens, posterior_matrix):
        "Accumulate counts of translations from: posterior_matrix[j][i] = p(a_j=i|e, f)"
        assert posterior_matrix.shape == (len(trg_tokens), len(src_tokens))
        for i, w1 in enumerate(src_tokens):
            for j, w2 in enumerate(trg_tokens):
                self._src_trg_counts[w1][w2] += posterior_matrix[j][i]
        
    def recompute_parameters(self):
        "Reestimate parameters and reset counters."
        self._trg_given_src_probs = defaultdict(lambda : defaultdict(lambda : 0.0))
        for w1 in self._src_trg_counts:
            sum_ = sum(self._src_trg_counts[w1].values())
            for w2, value in self._src_trg_counts[w1].items():
                self._trg_given_src_probs[w1][w2] += value/sum_
        
        self._src_trg_counts = defaultdict(lambda : defaultdict(lambda : 0.0))

class PriorModel:
    "Models the prior probability of an alignment given only the sentence lengths and token indices."

    use_distance = False
    normalize_distance = False
    dim = 10
    
    def __init__(self, src_corpus, trg_corpus):
        "Add counters and parameters here for more sophisticated models."
        if not self.normalize_distance:
            self._distance_counts = {}
            self._distance_probs = {}
        else:
            self._distance_counts = np.zeros((self.dim, self.dim))
            self._distance_probs = np.ones((self.dim, self.dim))/self.dim

    def _clamp(self, index, length):
        return int(index/length*self.dim)
            
    def get_parameters_for_sentence_pair(self, src_length, trg_length):
        key = (src_length, trg_length)
        if not self.use_distance or not self.normalize_distance and key not in self._distance_probs:
            return np.ones((src_length, trg_length)) * 1.0 / src_length
        
        if not self.normalize_distance:
            return self._distance_probs[key]
        
        result = np.zeros((src_length, trg_length))
        for i in range(src_length):
            for j in range(trg_length):
                src_index = self._clamp(i, src_length)
                trg_index = self._clamp(j, trg_length)
                result[i][j] = self._distance_probs[src_index][trg_index]
                
        return result
    
    def get_prior_prob(self, src_index, trg_index, src_length, trg_length):
        "Returns a uniform prior probability."
        key = (src_length, trg_length)
        if not self.use_distance or not self.normalize_distance and key not in self._distance_probs:
            return 1.0 / src_length
        
        if not self.normalize_distance:
            return self._distance_probs[key][src_index][trg_index]

        return self._distance_probs[self._clamp(src_index, src_length)][self._clamp(trg_index, trg_length)]

    def collect_statistics(self, src_length, trg_length, posterior_matrix):
        "Extract the necessary statistics from this matrix if needed."
        if not self.use_distance:
            return
        
        if not self.normalize_distance:
            key = (src_length, trg_length)
            if key not in self._distance_counts:
                self._distance_counts[key] = np.zeros(key)

            self._distance_counts[key] += posterior_matrix.T
            return

        for i in range(src_length):
            for j in range(trg_length):
                src_index = self._clamp(i, src_length)
                trg_index = self._clamp(j, trg_length)
                self._distance_counts[src_index][trg_index] += posterior_matrix[j][i]
        
    def recompute_parameters(self):
        "Reestimate the parameters and reset counters."
        if not self.use_distance:
            return
        
        if not self.normalize_distance:
            for key in self._distance_counts:
                denoms = np.sum(self._distance_counts[key], axis=0, keepdims=True)
                self._distance_probs[key] = self._distance_counts[key]/denoms

            self._distance_counts = {}
            return
        
        denoms = np.sum(self._distance_counts, axis=0, keepdims=True)
        self._distance_probs = self._distance_counts/denoms
        self._distance_counts = np.zeros((self.dim, self.dim))

class TransitionModel:
    "Models the prior probability of an alignment conditioned on previous alignment."

    def __init__(self, src_corpus, trg_corpus):
        "Add counters and parameters here for more sophisticated models."
        pass

    def get_parameters_for_sentence_pair(self, src_length):
        "Retrieve the parameters for this sentence pair: A[k, i] = p(a_{j} = i|a_{j-1} = k)"
        pass

    def collect_statistics(self, src_length, bigram_posteriors):
        "Extract statistics from the bigram posterior[i][j]: p(a_{t-1} = i, a_{t} = j| e, f)"
        pass
        
    def recompute_parameters(self):
        "Recompute the transition matrix"
        pass

In [2]:
!wget https://github.com/michmech/lemmatization-lists/raw/master/lemmatization-en.txt
!wget https://github.com/michmech/lemmatization-lists/raw/master/lemmatization-cs.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
def load_lemmas(file):
    lemmas = {}
    with open(file, encoding='utf8') as f:
        for line in f:
            lemma, word = line.strip().split('\t')
            lemmas[word] = lemma
    return lemmas

lemmas_en = load_lemmas('lemmatization-en.txt')
lemmas_cs = load_lemmas('lemmatization-cs.txt')

In [4]:
# This cell contains the framework for training and evaluating a model using EM.

from utils import read_parallel_corpus, extract_test_set_alignments, score_alignments, write_aligned_corpus
from itertools import chain

def normalize_word(word, lemmas, n_trim):
    word = word.lower()
    if word not in lemmas:
        return word[:n_trim]
    return lemmas[word][:n_trim]

def normalize_sentence(sentence, lemmas, n_trim):
    return list(map(lambda x: normalize_word(x, lemmas, n_trim), sentence))

def infer_posteriors(src_tokens, trg_tokens, prior_model, translation_model):
    "Compute the posterior probability p(a_j=i | f, e) for each target token f_j given e and f."
    # HINT: An HMM will require more complex statistics over the hidden alignments.
    P = prior_model.get_parameters_for_sentence_pair(len(src_tokens), len(trg_tokens)) # p[i][j] = P(a_ij)
    T = translation_model.get_parameters_for_sentence_pair(src_tokens, trg_tokens) # t[i][j] = P(f_j|e_i)
    posteriors = P*T
    marginals = np.sum(posteriors, axis=0)
    posteriors /= marginals
    log_likelihood = np.sum(np.log(marginals))
    return posteriors.T, log_likelihood

def collect_expected_statistics(src_corpus, trg_corpus, prior_model, translation_model):
    "E-step: infer posterior distribution over each sentence pair and collect statistics."
    corpus_log_likelihood = 0.0
    for src_tokens, trg_tokens in zip(src_corpus, trg_corpus):
        # Infer posterior
        posteriors, log_likelihood = infer_posteriors(src_tokens, trg_tokens, prior_model, translation_model)
        # Collect statistics in each model.
        prior_model.collect_statistics(len(src_tokens), len(trg_tokens), posteriors)
        translation_model.collect_statistics(src_tokens, trg_tokens, posteriors)
        # Update log prob
        corpus_log_likelihood += log_likelihood
    return corpus_log_likelihood

def estimate_models(src_corpus, trg_corpus, prior_model, translation_model, num_iterations):
    "Estimate models iteratively using EM."
    for iteration in range(num_iterations):
        # E-step
        corpus_log_likelihood = collect_expected_statistics(src_corpus, trg_corpus, prior_model, translation_model)
        # M-step
        prior_model.recompute_parameters()
        translation_model.recompute_parameters()
        if iteration > 0:
            print("corpus log likelihood: %1.3f" % corpus_log_likelihood)
    return prior_model, translation_model

def get_alignments_from_posterior(posteriors):
    "Returns the MAP alignment for each target word given the posteriors."
    # HINT: If you implement an HMM, you may want to implement a better algorithm here.
    alignments = {}
    for src_index, trg_index in enumerate(np.argmax(posteriors, 1)):
        if src_index not in alignments:
            alignments[src_index] = {}
        alignments[src_index][trg_index] = '*'
    return alignments

def align_corpus(src_corpus, trg_corpus, prior_model, translation_model):
    "Align each sentence pair in the corpus in turn."
    aligned_corpus = []
    for src_tokens, trg_tokens in zip(src_corpus, trg_corpus):
        posteriors, _ = infer_posteriors(src_tokens, trg_tokens, prior_model, translation_model)
        alignments = get_alignments_from_posterior(posteriors)
        aligned_corpus.append((src_tokens, trg_tokens, alignments))
    return aligned_corpus

def initialize_models(src_corpus, trg_corpus):
    prior_model = PriorModel(src_corpus, trg_corpus)
    translation_model = TranslationModel(src_corpus, trg_corpus)
    return prior_model, translation_model

def normalize(src_corpus, trg_corpus, norm, n_params, n_trim):
    if norm:
        normalized_src = list(map(lambda x: normalize_sentence(x, lemmas_en, n_trim), src_corpus))
        normalized_trg = list(map(lambda x: normalize_sentence(x, lemmas_cs, n_trim), trg_corpus))

        if n_params:
            src_unique = set(chain.from_iterable(normalized_src))
            trg_unique = set(chain.from_iterable(normalized_trg))
            
            src_mapping = {word: i%n_params for i, word in enumerate(src_unique)}
            trg_mapping = {word: i%n_params for i, word in enumerate(trg_unique)}
            
            hash_src = lambda src_sentence: list(map(lambda word: src_mapping[word]%n_params, src_sentence))
            hash_trg = lambda trg_sentence: list(map(lambda word: trg_mapping[word]%n_params, trg_sentence))
            
            normalized_src = list(map(hash_src, normalized_src))
            normalized_trg = list(map(hash_trg, normalized_trg))
    
        return normalized_src, normalized_trg
    return src_corpus, trg_corpus

def train(num_iterations, norm=False, n_params=None, n_trim=None):
    src_corpus, trg_corpus, _ = read_parallel_corpus('en-cs.all')
    src_corpus, trg_corpus = normalize(src_corpus, trg_corpus, norm, n_params, n_trim)
    prior_model, translation_model = initialize_models(src_corpus, trg_corpus)
    prior_model, translation_model = estimate_models(src_corpus, trg_corpus, prior_model, translation_model, num_iterations)    
    aligned_corpus = align_corpus(src_corpus, trg_corpus, prior_model, translation_model)
    return aligned_corpus, extract_test_set_alignments(aligned_corpus)

def evaluate(candidate_alignments):
    src_dev, trg_dev, wa_dev = read_parallel_corpus('en-cs-wa.dev', has_alignments=True)
    src_test, trg_test, wa_test = read_parallel_corpus('en-cs-wa.test', has_alignments=True)
    print('recall %1.3f; precision %1.3f; aer %1.3f' % score_alignments(wa_dev, candidate_alignments['dev']))
    print('recall %1.3f; precision %1.3f; aer %1.3f' % score_alignments(wa_test, candidate_alignments['test']))            

Сначала посмотрим на результат с обычным PriorModel и без нормализации текста

In [13]:
aligned_corpus, test_alignments = train(5)
evaluate(test_alignments)

corpus log likelihood: -1381305.485
corpus log likelihood: -1234633.856
corpus log likelihood: -1169520.317
corpus log likelihood: -1143343.672
recall 0.483; precision 0.434; aer 0.544
recall 0.480; precision 0.426; aer 0.550


Нормализуем текст (лемматизация)

In [14]:
aligned_corpus, test_alignments = train(5, norm=True)
evaluate(test_alignments)

corpus log likelihood: -1356617.212
corpus log likelihood: -1184343.907
corpus log likelihood: -1112444.266
corpus log likelihood: -1087090.923
recall 0.567; precision 0.506; aer 0.466
recall 0.564; precision 0.498; aer 0.472


Используем IBM Model 2

In [15]:
PriorModel.use_distance = True
aligned_corpus, test_alignments = train(5, norm=True)
evaluate(test_alignments)

corpus log likelihood: -1356617.212
corpus log likelihood: -1049028.146
corpus log likelihood: -833314.033
corpus log likelihood: -724005.932
recall 0.592; precision 0.535; aer 0.439
recall 0.600; precision 0.536; aer 0.435


Используем IBM Model 2 с нормализацей расстояния, в этой модели мы избавляемся от зависимости вероятностей от длины предложений

In [16]:
PriorModel.use_distance = True
PriorModel.normalize_distance = True
aligned_corpus, test_alignments = train(5, norm=True)
evaluate(test_alignments)

corpus log likelihood: -1077662.585
corpus log likelihood: -875468.871
corpus log likelihood: -744753.776
corpus log likelihood: -673244.557
recall 0.664; precision 0.606; aer 0.367
recall 0.673; precision 0.607; aer 0.363


Для каждого слова при нормализации оставим только $n$ букв. Сначала хотел сделать это только для тех слов, для которых не нашлось нормальной формы в словарике, но оказалось, что качество улучшается если проделать это для всех слов. Далее подберем такое n, при котором получим наименьший AER

In [18]:
PriorModel.use_distance = True
PriorModel.normalize_distance = True
aligned_corpus, test_alignments = train(5, norm=True, n_trim=4)
evaluate(test_alignments)

corpus log likelihood: -1095914.553
corpus log likelihood: -897770.427
corpus log likelihood: -764167.632
corpus log likelihood: -694624.096
recall 0.714; precision 0.651; aer 0.320
recall 0.710; precision 0.640; aer 0.328


In [19]:
PriorModel.use_distance = True
PriorModel.normalize_distance = True
aligned_corpus, test_alignments = train(5, norm=True, n_trim=5)
evaluate(test_alignments)

corpus log likelihood: -1086179.937
corpus log likelihood: -875310.607
corpus log likelihood: -736549.810
corpus log likelihood: -666861.825
recall 0.710; precision 0.647; aer 0.324
recall 0.711; precision 0.641; aer 0.327


In [23]:
PriorModel.use_distance = True
PriorModel.normalize_distance = True
aligned_corpus, test_alignments = train(5, norm=True, n_trim=6)
evaluate(test_alignments)

corpus log likelihood: -1081635.925
corpus log likelihood: -870495.182
corpus log likelihood: -732633.979
corpus log likelihood: -662343.348
recall 0.697; precision 0.635; aer 0.337
recall 0.702; precision 0.632; aer 0.336


Попробуем при нормализации еще уменьшить количество параметров

In [24]:
PriorModel.use_distance = True
PriorModel.normalize_distance = True
aligned_corpus, test_alignments = train(5, norm=True, n_params=1000, n_trim=5)
evaluate(test_alignments)

corpus log likelihood: -1138015.153
corpus log likelihood: -1014288.493
corpus log likelihood: -906801.819
corpus log likelihood: -834761.967
recall 0.681; precision 0.627; aer 0.349
recall 0.683; precision 0.620; aer 0.352


In [21]:
PriorModel.use_distance = True
PriorModel.normalize_distance = True
aligned_corpus, test_alignments = train(5, norm=True, n_params=5000, n_trim=5)
evaluate(test_alignments)

corpus log likelihood: -1106181.472
corpus log likelihood: -903101.507
corpus log likelihood: -765358.626
corpus log likelihood: -695055.993
recall 0.705; precision 0.643; aer 0.329
recall 0.708; precision 0.638; aer 0.330


In [25]:
PriorModel.use_distance = True
PriorModel.normalize_distance = True
aligned_corpus, test_alignments = train(5, norm=True, n_params=10000, n_trim=5)
evaluate(test_alignments)

corpus log likelihood: -1085561.621
corpus log likelihood: -875220.415
corpus log likelihood: -736552.567
corpus log likelihood: -666811.046
recall 0.710; precision 0.647; aer 0.324
recall 0.711; precision 0.640; aer 0.327


Изменение `n_params` не дало прироста качества. Попробуем теперь поменять параметр `dim` в PriorModel

In [27]:
PriorModel.use_distance = True
PriorModel.normalize_distance = True
PriorModel.dim = 20
aligned_corpus, test_alignments = train(5, norm=True, n_trim=5)
evaluate(test_alignments)

corpus log likelihood: -1263342.805
corpus log likelihood: -1043689.705
corpus log likelihood: -896492.498
corpus log likelihood: -823352.346
recall 0.713; precision 0.651; aer 0.321
recall 0.718; precision 0.645; aer 0.322


In [28]:
PriorModel.use_distance = True
PriorModel.normalize_distance = True
PriorModel.dim = 25
aligned_corpus, test_alignments = train(5, norm=True, n_trim=5)
evaluate(test_alignments)

corpus log likelihood: -1321161.002
corpus log likelihood: -1099956.638
corpus log likelihood: -951305.541
corpus log likelihood: -878282.629
recall 0.714; precision 0.653; aer 0.319
recall 0.722; precision 0.649; aer 0.318


In [29]:
PriorModel.use_distance = True
PriorModel.normalize_distance = True
PriorModel.dim = 30
aligned_corpus, test_alignments = train(5, norm=True, n_trim=5)
evaluate(test_alignments)

corpus log likelihood: -1365392.053
corpus log likelihood: -1142171.810
corpus log likelihood: -992404.852
corpus log likelihood: -920019.406
recall 0.713; precision 0.651; aer 0.321
recall 0.717; precision 0.646; aer 0.322


В итоге достигли `AER=0.318`

In [32]:
# To visualize aligned corpus:
# 1. call write_aligned_corpus(aligned_corpus, 'out')
# 2. run python corpus_browser.py en-cs-wa.out (in working directory)

write_aligned_corpus(aligned_corpus, 'out')
!python corpus_reader.py en-cs-wa.out

python: can't open file 'corpus_reader.py': [Errno 2] No such file or directory


In [33]:
# Discrete HMM with scaling. You may want to use this if you decide to implement an HMM.
# The parameters for this HMM will still need to be provided by the models above.

def forward(pi, A, O):
    S, T = O.shape
    alpha = np.zeros((S, T))
    scaling_factors = np.zeros(T)
    
    # base case
    alpha[:, 0] = pi * O[:, 0]
    scaling_factors[0] = np.sum(alpha[:, 0])
    alpha[:, 0] /= scaling_factors[0] 
    
    # recursive case
    for t in range(1, T):
        alpha[:, t] = np.dot(alpha[:, t-1], A[:, :]) * O[:, t]

        # Normalize at each step to prevent underflow.
        scaling_factors[t] = np.sum(alpha[:, t])
        alpha[:, t] /= scaling_factors[t]

    return (alpha, scaling_factors)

def backward(pi, A, O, forward_scaling_factors):
    S, T = O.shape
    beta = np.zeros((S, T))

    # base case
    beta[:, T-1] = 1 / forward_scaling_factors[T-1]
    
    # recursive case
    for t in range(T-2, -1, -1):
        beta[:, t] = np.sum(beta[:, t+1] * A[:, :] * O[:, t+1], 1) / forward_scaling_factors[t]

    return beta

def forward_backward(pi, A, O):
    alpha, forward_scaling_factors = forward(pi, A, O)
    beta = backward(pi, A, O, forward_scaling_factors)
    return alpha, beta, np.sum(np.log(forward_scaling_factors))
